# Notebook Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Constants

In [2]:
TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Load the Data

In [3]:
# Features
X_test = np.loadtxt(TEST_FEATURE_MATRIX, delimiter=' ')
# Target
y_test = np.loadtxt(TEST_TARGET_FILE, delimiter=' ')
# Token probabilities
prob_token_spam = np.loadtxt(TOKEN_SPAM_PROB_FILE, delimiter=' ')
prob_token_ham = np.loadtxt(TOKEN_HAM_PROB_FILE, delimiter=' ')
prob_all_tokens = np.loadtxt(TOKEN_ALL_PROB_FILE, delimiter=' ')

In [4]:
X_test.shape

(1724, 2500)

In [5]:
y_test.shape

(1724,)

# Calculating Joint Probability
## Dot Product

In [6]:
a = np.array([1, 2, 3])
b = np.array([0, 5, 4])
print('a =', a)
print('b =', b)

a = [1 2 3]
b = [0 5 4]


In [7]:
a.dot(b)

22

In [8]:
1*0 + 2*5 + 3*4

22

In [9]:
c = np.array([[0, 6], [3, 0], [5, 1]])
print('Shape of c is:', c.shape)
print(c)

Shape of c is: (3, 2)
[[0 6]
 [3 0]
 [5 1]]


In [10]:
print(a.dot(c))
print('Shape of dot product is:', a.dot(c).shape)

[21  9]
Shape of dot product is: (2,)


In [11]:
[1*0 + 2*3 + 3*5, 1*6 + 2*0 + 3*1]

[21, 9]

In [12]:
X_test.shape

(1724, 2500)

In [13]:
prob_token_spam.shape

(2500,)

In [14]:
print('Shape of dot product:', X_test.dot(prob_token_spam).shape)

Shape of dot product: (1724,)


## Set the Prior
$$ P(Spam \, | \, X) = \frac{P(X \, | \, Spam \,) \, P(Spam)}{P(X)}$$

In [15]:
PROB_SPAM = 0.310989

In [16]:
np.log(prob_token_spam)

array([ -4.40760872,  -5.25368353,  -4.99008596, ..., -10.30247059,
        -8.83613352, -12.09423006])

## Joint probability in log format

In [17]:
joint_log_spam = X_test.dot(np.log(prob_token_spam) - np.log(prob_all_tokens)) + np.log(PROB_SPAM)

In [18]:
joint_log_spam

array([  24.27862556,    2.1604211 ,   20.5904407 , ..., -374.66032958,
         -9.90362646, -112.01986139])

$$ P(Ham \, | \, X) = \frac{P(X \, | \, Ham \,) \, (1 - P(Spam))}{P(X)}$$

In [19]:
joint_log_ham = X_test.dot(np.log(prob_token_ham) - np.log(prob_all_tokens)) + np.log(1 - PROB_SPAM)

In [20]:
joint_log_ham

array([-6.09702401e+01, -1.10100708e+01, -3.79679398e+01, ...,
        6.09737116e+01, -5.82211832e-02,  2.44516960e+01])

In [21]:
joint_log_ham.size

1724

# Making Predictions
### Checking for Higher Joint Probability
$$ P(Spam \, | \, X) \, > \, P(Ham \, | \, x) $$
**<center>OR</center>**

$$ P(Spam \, | \, X) \, < \, P(Ham \, | \, x) $$

In [22]:
prediction = joint_log_spam > joint_log_ham

In [23]:
prediction * 1

array([1, 1, 1, ..., 0, 0, 0])

In [24]:
y_test

array([1., 1., 1., ..., 0., 0., 0.])

### Simplify

In [25]:
joint_log_spam = X_test.dot(np.log(prob_token_spam)) + np.log(PROB_SPAM)
joint_log_ham = X_test.dot(np.log(prob_token_ham)) + np.log(1 - PROB_SPAM)

# Metrix and Evaluation
## Accuracy

In [26]:
correct_docs = (y_test == prediction).sum()
print('Docs classified correctly:', correct_docs)
num_docs_wrong = X_test.shape[0] - correct_docs
print('Docs classified incorrectly:', num_docs_wrong)

Docs classified correctly: 1685
Docs classified incorrectly: 39


In [27]:
# Accuracy
correct_docs/len(X_test)

0.9773781902552204

In [28]:
fraction_worng = num_docs_wrong/len(X_test)
print('Fraction classified incorrectly is {:.2%}'.format(fraction_worng))
print('Accuracy of the model is {:.2%}'.format(1 - fraction_worng))

Fraction classified incorrectly is 2.26%
Accuracy of the model is 97.74%
